In [156]:

# Install required libraries
# Import required libraries
import random
import math
import gymnasium as gym
from gymnasium import spaces
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import namedtuple, deque
from itertools import count
import keras


device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "cpu"
)

print(device)
CUDA_LAUNCH_BLOCKING=1

cuda


In [157]:
# imports are always needed
import torch
# get index of currently selected device
torch.cuda.current_device() # returns 0 in my case
# get number of GPUs available
torch.cuda.device_count() # returns 1 in my case
# get the name of the device
torch.cuda.get_device_name(0) # good old Tesla K80


'NVIDIA GeForce RTX 4070 Laptop GPU'

In [158]:

# modify this to fit current environment 

def DQN(obs,actions):
    model = keras.Sequential()
    model.add(keras.layers.Dense(32, input_dim=obs, activation = 'relu'))
    model.add(keras.layers.Dense(32, activation = 'relu'))
    model.add(keras.layers.Dense(actions, activation = 'linear'))
    model.compile(optimizer = 'adam', loss = 'mse',metrics=['loss'])
    return model


    
class agent():
    def __init__(self,obs,actions):
        self.actions = actions
        self.batch = 128
        self.discount_factor = 0.99
        self.eps = 1
        self.eps_decay = 0.9954
        # self.tau = 0.005
        self.learning_rate = 0.1
        self.memory = deque([], maxlen=5000) # replay memory 
        # self.policy_net = DQN(obs, actions).to(device) # action value function
        # self.target_net = DQN(obs, actions).to(device) # target action value function 
        self.policy_net = DQN(obs, actions)
        self.target_net = DQN(obs, actions)
        self.target_net.set_weights(self.policy_net.get_weights())
        # self.Q_table = self.print_table() # not working rn
        
    
    def select_action(self,env,state):
        p = random.random()
        if p < self.eps:
            #random action
            return env.action_space.sample()
        else:
            #best action
            print(state)
            q_table = self.policy_net.predict(state,verbose = 0)[0]
            print(q_table)
            return np.amax(q_table)
    
    def replay(self):
        if len(agent.memory) < self.batch:
            return
        else:
            sample = random.sample(self.memory, self.batch)
            states_list = []
            q_values_list = []
            for state, action, reward, next_state,terminated in sample:
                if terminated == True:
                    target = reward
                else:
                    # next_q = self.target_net(next_state).max(1)
                    next_q = self.target_net.predict(next_state)[0]
                    target = (reward + self.learning_rate * np.amax(next_q))
                q_values = self.policy_net(state,verbose = 0)[0]
                q_values[action] = target
                states_list.append(state)
                q_values_list.append(q_values)
            self.policy_net.fit(states_list,q_values_list,epochs = 1,verbose = 0)
            return

            
        #replay algorithim here
    # def print_table(self): # not working rn
        
    #     return 
    

In [159]:
#create environment to run DQN
if __name__ == "__main__":
    
    env = gym.make("CartPole-v1")
    state, info = env.reset()
    obs = len(state)
    actions = env.action_space.n
    agent = agent(obs,actions)
    max_episodes = 600
    max_timestep = 500
    state, info = env.reset()
    obs = len(state)
    actions = env.action_space.n
    # agent = agent(obs,actions)
    reward_per_episode = []
    for episode in range(max_episodes):
        # Initialize the environment and get its state
        state, info = env.reset()
        if episode%5 == 0 or truncated == True:
            agent.target_net.set_weights(agent.policy_net.get_weights())
        terminated,truncated = False, False
        for timestep in range(500):
            action = agent.select_action(env,state)
            observation, reward, terminated, truncated, _ = env.step(action)
            # reward +=timestep
            next_state = np.reshape(observation, [1, obs])
            # reward = torch.tensor(reward, dtype=torch.float32).unsqueeze(0)
            q = [state, action, reward, next_state,terminated]
            agent.memory.append(q)
            # Move to the next state
            state = next_state
            if timestep % 3  == 0 :
                agent.replay() #update network weights
            if terminated or truncated: # close loop if it ends
                print('loop end at:', timestep)
                reward_per_episode.append(timestep)
                break
        agent.eps = agent.eps * agent.eps_decay #eps decay

    print('solved',truncated)
    plt.figure()
    fig ,  ax = plt.subplots()
    plt.xlabel('Episode', fontsize=20)
    plt.ylabel('Timestep', fontsize=20)
    plt.title('Cumulative Reward Per Episode', fontsize=24)
    ax.plot(reward_per_episode,linestyle='solid',label = 'Q learning')
    plt.grid()
    plt.show()

loop end at: 12
loop end at: 16
[[-0.04254026 -0.17621116  0.09576438  0.41936132]]
[-0.04353275 -0.00275847]


C:\Users\corvi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AssertionError: np.float32(-0.002758469) (<class 'numpy.float32'>) invalid